DenseNet121

Install Dependencies

In [1]:
%pip install torch torchvision
%pip install --upgrade torch
%pip install --upgrade torchvision
%pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


Run the Model

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
from PIL import Image
from tempfile import TemporaryDirectory
import shutil
from sklearn.model_selection import train_test_split

cudnn.benchmark = True
plt.ion()  

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}
data_dir = '..\data\separated_imgdata'
dataset = datasets.ImageFolder(root = data_dir, transform = data_transforms['train'])

train_size = int(0.7 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])
batch_size = 32

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
class_names = dataset.classes

print("Classes: ", dataset.classes)
print(f"Training size: {len(train_dataset)}")
print(f"Test size: {len(test_dataset)}")
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model_ft = models.densenet121(weights='IMAGENET1K_V1')
num_ftrs = model_ft.classifier.in_features
model_ft.classifier = nn.Linear(num_ftrs, len(class_names))

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = model.state_dict()
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch + 1}/{num_epochs}')
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train() 
            else:
                model.eval()  

            running_loss = 0.0
            running_corrects = 0

            dataloader = train_loader if phase == 'train' else test_loader
            for inputs, labels in dataloader:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / len(train_dataset if phase == 'train' else test_dataset)
            epoch_acc = running_corrects.double() / len(train_dataset if phase == 'train' else test_dataset)

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = model.state_dict()

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:.4f}')

    model.load_state_dict(best_model_wts)
    return model

model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=25)

Image file not found: ..\data\80-20\Training 70\1.png
Image file not found: ..\data\80-20\Training 70\2.png
Image file not found: ..\data\80-20\Training 70\3.png
Image file not found: ..\data\80-20\Training 70\3.1.png
Image file not found: ..\data\80-20\Training 70\4.png
Image file not found: ..\data\80-20\Training 70\5.png
Image file not found: ..\data\80-20\Training 70\6.png
Image file not found: ..\data\80-20\Training 70\6.1.png
Image file not found: ..\data\80-20\Training 70\6.2.png
Image file not found: ..\data\80-20\Training 70\7.png
Image file not found: ..\data\80-20\Training 70\8.png
Image file not found: ..\data\80-20\Training 70\9.png
Image file not found: ..\data\80-20\Training 70\9.1.png
Image file not found: ..\data\80-20\Training 70\10.png
Image file not found: ..\data\80-20\Training 70\11.png
Image file not found: ..\data\80-20\Training 70\12.png
Image file not found: ..\data\80-20\Training 70\13.png
Image file not found: ..\data\80-20\Training 70\14.png
Image file not 

FileNotFoundError: Found no valid file for the classes Bug, Dark, Dragon, Electric, Fairy, Fighting, Fire, Flying, Ghost, Grass, Ground, Ice, Normal, Poison, Psychic, Rock, Steel, Water. Supported extensions are: .jpg, .jpeg, .png, .ppm, .bmp, .pgm, .tif, .tiff, .webp

Evaluate The Model

In [ ]:
from sklearn.metrics import classification_report
import torch

model_ft.eval()

all_labels = []
all_preds = []

with torch.no_grad(): 
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model_ft(inputs)
        _, preds = torch.max(outputs, 1)

        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(preds.cpu().numpy())

report = classification_report(all_labels, all_preds, target_names=class_names)

print(report)

ModuleNotFoundError: No module named 'tensorflow.keras'